In [130]:
#download and install CBC solver
#%pip install pyomo-windows

# from pyomo_windows.solvers import DownloadSolvers
# downloader = DownloadSolvers()
# downloader.download_cbc()

In [131]:
import matplotlib.pyplot as plt
import pandas as pd
import importlib
from pyomo.environ import *
%matplotlib inline

import MultiCriteriaMIPModel as MultiCriteriaMIPModel_Module
importlib.reload(MultiCriteriaMIPModel_Module) # in case of updates

from MultiCriteriaMIPModel import MultiCriteriaMIPModel


In [132]:
mcmModel= MultiCriteriaMIPModel()
#instance, results = mcmModel.solve("Mip_parameters.dat", "cbc")
instance, results = mcmModel.solve("Mip_parameters.dat")

In [133]:
mcmModel.display_solution(instance)

Model unknown

  Variables:
    y : Size=2, Index=I_max
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          A :     0 :   1.0 :     1 : False : False : Binary
          B :     0 :   1.0 :     1 : False : False : Binary
    z : Size=50, Index=I_max*J_prime*J_prime
        Key         : Lower : Value : Upper : Fixed : Stale : Domain
        ('A', 0, 0) :     0 :  None :     1 : False :  True : Binary
        ('A', 0, 1) :     0 :   0.0 :     1 : False : False : Binary
        ('A', 0, 2) :     0 :   0.0 :     1 : False : False : Binary
        ('A', 0, 3) :     0 :   0.0 :     1 : False : False : Binary
        ('A', 0, 4) :     0 :   1.0 :     1 : False : False : Binary
        ('A', 1, 0) :     0 :   1.0 :     1 : False : False : Binary
        ('A', 1, 1) :     0 :  None :     1 : False :  True : Binary
        ('A', 1, 2) :     0 :   0.0 :     1 : False : False : Binary
        ('A', 1, 3) :     0 :   0.0 :     1 : False : False : Binary
        ('A', 1, 4) :     

In [134]:
solution_data = []
for i in instance.I_max:
    if value(instance.y[i]) == 1: #in case of unfeasible solution, it launches a pyomo error
        #operator i is active
        for j in instance.J_prime:
            for k in instance.J_prime:
                
                #apply the j != k filter to avoid pyomo warning about self-loops
                if j != k:
                    #check if the index (i, j, k) is valid and exists in z
                    if (i, j, k) in instance.z:
                        
                        #check for active flow
                        if value(instance.z[i, j, k]) > 0.5:
                            
                            #add data only for flow between service orders (not flow to/from Base)
                            if j in instance.J and k in instance.J:
                                solution_data.append({
                                    'Operator': i,
                                    'Task': j,
                                    'Start': value(instance.S[j]),
                                    'Finish': value(instance.C[j]),
                                    'Successor': k
                                })

df_schedule = pd.DataFrame(solution_data)
df_schedule = df_schedule.sort_values(by=['Operator', 'Start']).reset_index(drop=True)
print(df_schedule)

  Operator  Task  Start  Finish  Successor
0        A     4    0.0    18.0          1
1        B     2    0.0    10.0          3


In [135]:
routes = {} 
    
# Check for solution status (optional but recommended)
# if not (results.solver.status == SolverStatus.ok and results.solver.termination_condition == TerminationCondition.optimal):
#     print("Solver did not find an optimal solution.")
#     return

# Call the function with your solved instance
# print_optimal_routes(instance)

# 1. Iterate over all potential operators
for i in instance.I_max:
    if value(instance.y[i]) < 0.5: #in case of unfeasible solution, it launches a pyomo error
        continue #skip unactivated operators

    print(f"\n--- Operator {i} (Activated) ---")

    current_node = 0  # Start at the Base node
    route_sequence = []
    is_route_complete = False

    #security counter to prevent infinite loops (should not happen if flow constraints are correct)
    max_steps = len(instance.J) + 2 
    steps = 0

    #loop until the route returns to the Base (k=0)
    while not is_route_complete and steps < max_steps:
        steps += 1
        
        #search for the next step (k) starting from the current node (current_node)
        found_next_step = False
        for k in instance.J_prime:
            if current_node == k:
                continue # Skip self-loop
            
            try:
                #check if the arc (current_node -> k) is active
                if value(instance.z[i, current_node, k]) > 0.5:
                    
                    #calculate travel time for printing
                    travel_time = value(instance.T[current_node, k])
                    
                    #handle Movement Types
                    if k == 0:
                        #final movement: Return to Base
                        route_sequence.append(f"-> Base (Arc: {current_node} -> 0 | Travel: {travel_time:.2f})")
                        is_route_complete = True
                        break #exit the k loop
                    else:
                        #service movement: j -> k
                        start_time = value(instance.S[k])
                        finish_time = value(instance.C[k])
                        proc_time = finish_time - start_time
                        
                        movement_detail = (
                            f"-> Order {k} | Travel: {travel_time:.2f} | Start: {start_time:.2f} | "
                            f"Proc: {proc_time:.2f} | Finish: {finish_time:.2f}"
                        )
                        route_sequence.append(movement_detail)
                        
                        #move to the next node in the sequence
                        current_node = k
                        found_next_step = True
                        break #exit the k loop
                        
            except KeyError:
                #this node pair might not exist in the defined set of z variables (e.g., if filtered by a complex index)
                continue

        if not found_next_step and not is_route_complete:
            print(f"Error: Route stopped unexpectedly at node {current_node} for operator {i}.")
            break
        
    #print the final sequenced route for the operator
    route_string = "\n".join(route_sequence)
    print("Path: Base " + route_string)
    print(f"Total Time (C_last): {value(instance.C_last[i]):.2f}")
    print("-" * 40)


--- Operator A (Activated) ---
Path: Base -> Order 4 | Travel: 7.00 | Start: 0.00 | Proc: 18.00 | Finish: 18.00
-> Order 1 | Travel: 2.00 | Start: 20.00 | Proc: 20.00 | Finish: 40.00
-> Base (Arc: 1 -> 0 | Travel: 6.00)
Total Time (C_last): 46.00
----------------------------------------

--- Operator B (Activated) ---
Path: Base -> Order 2 | Travel: 8.00 | Start: 0.00 | Proc: 10.00 | Finish: 10.00
-> Order 3 | Travel: 5.00 | Start: 15.00 | Proc: 20.00 | Finish: 35.00
-> Base (Arc: 3 -> 0 | Travel: 5.00)
Total Time (C_last): 46.00
----------------------------------------
